# Spotify Music Popularity Capstone - Data Collection & Wrangling

Greg Welliver   

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.ticker as plticker
import seaborn as sns
sns.set()
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats

In [3]:
# load the dataframe
df1 = pd.read_csv("../Data/Merged_Data7.csv", index_col=[0])

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255096 entries, 0 to 822121
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          255096 non-null  object 
 1   track_name        255096 non-null  object 
 2   track_genre       255096 non-null  object 
 3   artist_name       255096 non-null  object 
 4   track_popularity  255096 non-null  float64
 5   popular_year      255096 non-null  int64  
 6   Top100            255096 non-null  object 
 7   key               255096 non-null  float64
 8   acousticness      255096 non-null  float64
 9   danceability      255096 non-null  float64
 10  duration_ms       255096 non-null  float64
 11  energy            255096 non-null  float64
 12  instrumentalness  255096 non-null  float64
 13  liveness          255096 non-null  float64
 14  loudness          255096 non-null  float64
 15  modality          255096 non-null  float64
 16  speechiness       25

# previous top 100 section

This section is intended to decipher if an artist has previously had a song in a Top Hits playlist on Spotify.

In [6]:
df1['prevTop100'] = 'N'

#### what I need to accomplish:

for each row (track), determine if the artist of that track has previously had a track with a Top100 = "Y".  If this is true, then I want to populate the "prevTop100" column with a "Y". Note that this should only occur for Top100 songs that are not the first Top Hit for the artist (e.g. their first Top100 hit should have a "prevTop100" value of N).

In the Britney Spears example below, the first two tracks, which were in the year 2000, would have prevTop100 values of "N", while all of the tracks in 2001 and forward would have a prevTop100 value of "Y".
    


In [71]:
#df1[df1['artist_name'] == 'Eminem' & df1['Top100'] == "Y"]
display(df1.loc[(df1['artist_name'] == 'Britney Spears') & (df1['Top100'] == 'Y')])

,track_id,track_name,track_genre,artist_name,track_popularity,key,acousticness,danceability,duration_ms,energy,...,tempo,time_signature,valence,popular_year,Top100,second_artist,artist_country,Label,collaboration,prevTop100
33729,6naxalmIoLFWR0siv8dnQQ,Oops!...I Did It Again,pop,Britney Spears,78,1.0,0.30000,0.751,211160.0,0.834,...,95.053,4.0,0.894,2000,Y,NaN,United States,Jive,N,N
33819,2TTYIwTM2iLC1YOyHuhRMt,Lucky,pop,Britney Spears,67,8.0,0.26200,0.765,206227.0,0.791,...,95.026,4.0,0.966,2000,Y,NaN,United States,Jive,N,N
38079,6ldwfK0yWgTAlmIfuQkTYN,I'm a Slave 4 U,pop,Britney Spears,69,5.0,0.41500,0.847,203600.0,0.843,...,110.027,4.0,0.963,2001,Y,NaN,United States,Jive,N,N
38717,5p0KCAuu5nrHpTuAIhHdLw,Overprotected - Radio Edit,pop,Britney Spears,63,0.0,0.03810,0.682,198600.0,0.894,...,95.992,4.0,0.845,2002,Y,NaN,United States,Jive,N,N
39198,1xg1yGPy1Y3YUf44s6TB1o,"I'm Not a Girl, Not Yet a Woman",pop,Britney Spears,60,3.0,0.57900,0.534,231067.0,0.543,...,78.996,4.0,0.418,2002,Y,NaN,United States,Jive,N,N
146115,6I9VzXrHxO9rA9A5euc8Ak,Toxic,pop,Britney Spears,82,5.0,0.02490,0.774,198800.0,0.838,...,143.040,4.0,0.924,2004,Y,NaN,United States,Jive,N,N
146831,7mS8RbJji2UZAaguRGsOCH,Me Against the Music (feat. Madonna) - LP Vers...,pop,Britney Spears,60,6.0,0.32000,0.804,223773.0,0.836,...,120.046,4.0,0.850,2003,Y,NaN,United States,Jive,Y,N
358656,6ic8OlLUNEATToEFU3xmaH,Gimme More,pop,Britney Spears,79,2.0,0.25000,0.788,251240.0,0.844,...,113.324,4.0,0.382,2007,Y,NaN,United States,Jive,N,N
359239,2EvwLVrnYbCZEG6Kx5DCRy,Piece of Me,pop,Britney Spears,64,11.0,0.09020,0.769,212107.0,0.638,...,115.007,4.0,0.782,2008,Y,NaN,United States,Jive,N,N
359375,52K4Nl7eVNqUpUeJeWJlwT,Break the Ice,pop,Britney Spears,61,5.0,0.68900,0.712,196053.0,0.911,...,117.533,4.0,0.850,2008,Y,NaN,United States,Jive,N,N


In [19]:
# create the dataframe to compare against
top100df = df1[df1['Top100'] == 'Y']
top100df.head(4)

,track_id,track_name,track_genre,artist_name,track_popularity,popular_year,Top100,key,acousticness,danceability,...,modality,speechiness,tempo,time_signature,valence,second_artist,artist_country,collaboration,prevTop100,Label
0,3yfqSUWxFvZELEM4PmlwIR,The Real Slim Shady,hip-hop,Eminem,88.0,2000,Y,5.0,0.0302,0.949,...,0.0,0.0572,104.504,4.0,0.760,NaN,United States,N,N,"Aftermath, Interscope, Shady"
12,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,hip-hop,Outkast,82.0,2001,Y,4.0,0.1430,0.843,...,0.0,0.2690,94.948,4.0,0.613,NaN,NaN,N,N,NaN
13,3UmaczJpikHgJFyBTAJVoz,Stan,hip-hop,Eminem,85.0,2001,Y,6.0,0.0371,0.780,...,0.0,0.2380,80.063,4.0,0.507,NaN,United States,N,N,"Aftermath, Interscope, Shady"
25,3Gf5nttwcX9aaSQXRWidEZ,Ride Wit Me,hip-hop,Nelly,75.0,2001,Y,7.0,0.0616,0.850,...,1.0,0.0478,101.875,4.0,0.722,NaN,United States,N,N,Universal Motown


#### Part 1

In [21]:
# in this first for loop, I'm trying to create a "temporary" dataframe that has each artist with a Top100 hit, and specifically find the year of their first Top100 hit
for artist in top100df['artist_name']:
    #temp_df = (top100df[top100df['artist_name'] == artist] & top100df[top100df['popular_year']].min())
    temp_df = df1.loc[(df1['artist_name'] == artist) & (df1['popular_year'].min())]
    print(temp_df)

# when you run this code, you'll see that it currently creates a series of empty dataframes

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

...

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]
Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist

Empty DataFrame
Columns: [track_id, track_name, track_genre, artist_name, track_popularity, popular_year, Top100, key, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, modality, speechiness, tempo, time_signature, valence, second_artist, artist_country, collaboration, prevTop100, Label]
Index: []

[0 rows x 25 columns]


#### Part 2: once I'm able to get that for loop right, I then need to try to compare the current year to the year that I isolated for in the previous for loop

In [ ]:
# just scribbling here for now
for row in df1['prev_Top100']:
    if df1['popular_year'] > ("value returned from previous for loop")
        df1['prev_Top100'] = "Y"
    else:
        df1['prev_Top100'] = "N"


#### subset dataframe on the fly approach


#### create dictionary approach
almost like if for each artist, you created a dictionary of years that they previously had a top 100, you could look up that list

one approach: 
embedded for loop where you look at previous dataframe

another approach:
join between two dataframes



#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Capstone Two - Music/Data/Merged_Data7.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df7.to_csv(filepath)

### Reference docs

spotipy documentation
https://spotipy.readthedocs.io/en/latest/#installation

google sheet with links to top song playlists: https://docs.google.com/spreadsheets/d/10gRoHKIB1RFfxSvb83gzYkV33x5eMMI1RZoXSamFwPI/edit#gid=824975682

github for project that I referenced to pull data: https://github.com/ekatnic/SpotifyApiExploration/blob/master/Spotify_Generate_Tracks.ipynb

Proposal: https://docs.google.com/document/d/1rRzJV0hOaNMsBIjji8v1Cv8C3QUuZ6lUvYokZ5dif-A/edit